In [2]:
pip install pecanpy

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/24.0 MB 5.9 MB/s eta 0:00:05
    --------------------------------------- 0.4/24.0 MB 6.8 MB/s eta 0:00:04
    --------------------------------------- 0.5/24.0 MB 3.1 MB/s eta 0:00:08
   - -------------------------------------- 0.9/24.0 MB 5.0 MB/s eta 0:00:05
   - -------------------------------------- 1.1/24.0 MB 5.0 MB/s eta 0:00:05
   -- ------------------------------------- 1.3/24.0 MB 4.3 MB/s eta 0:00:06
   -- ------------------------------------- 1.6/24.0 MB 4.7 MB/s eta 0:00:05
   --- ------------------------------------ 1.9/24.0 MB 4.8 MB/s eta 0:00:05
   --- ------------------------------------ 2.3/24.0 MB 5.2 MB/s eta 0:00:05
   ---- ----------------------------------- 2.7/24.0 MB 5.4 MB/s eta 0:00:04
   ----- ---------------------------------- 3.1/24.0 MB 5.7 MB/s eta 0:00:04
   ----- ---------------------------------- 3.5/24.0 MB 5.8 MB/s eta 0:00:04
   ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
# Importing libraries and defining constants

import pandas as pd
import numpy as np
from pecanpy import node2vec
from gensim.models import Word2Vec

from Constants import Constants
from tqdm import tqdm
from joblib import load, dump

GRAPH_FILE_NAME = Constants.GRAPH_FILE_NAME.value
WEIGHTED = Constants.WEIGHTED.value
DIRECTED = Constants.DIRECTED.value
NUM_WALK = Constants.NUM_WALK.value
WALK_LEN = Constants.WALK_LEN.value

print(GRAPH_FILE_NAME, NUM_WALK, WALK_LEN)

In [2]:
# Loading the graph data from a parquet file

graph_path = '../Data/ConstructedGraph/{}'.format(GRAPH_FILE_NAME)
graph = pd.read_parquet(graph_path)

print("Graph Read from {}".format(graph_path))

Graph Read from ../Data/ConstructedGraph/undirected_weighted_product_views_graph.parquet


In [3]:
graph

,pid_1,pid_2,occurence_ct
0,3601512,3601269,37
1,100059274,3601306,12
2,5100855,4804056,880
3,100041977,100041934,2
4,100057579,100057560,11
...,...,...,...
6846587,12300752,4803895,1
6846588,100017063,6501098,1
6846589,8800741,1005195,1
6846590,25200369,23900098,1


In [4]:
# Calculate and display the count of nodes and edges in the graph

Node_Count = len(set(graph['pid_1'].unique()).union(graph['pid_2'].unique()))
Edge_Count = len(graph)

print("Graph {} has Nodes: {} | Edges: {}".format(GRAPH_FILE_NAME, Node_Count, Edge_Count))

Graph undirected_weighted_product_views_graph.parquet has Nodes: 211861 | Edges: 6846592


In [6]:
# Saving Graph in .edg format

edg_graph_path = '../Data/Edg_Graphs_DataFile/'+GRAPH_FILE_NAME.split('.')[0]+'.edg'
graph.to_csv(edg_graph_path, sep='\t', index=False, header=False)
print("Edg Graph Saved")

### Graph Random Walk Generation

In [6]:
# Initialize the node2vec model with parameters for Deepwalk
# Deepwalk has p=1 and q=1 (BFS and DFS equally important)

g = node2vec.SparseOTF(p=1, q=1, workers=-1, verbose=True, extend=False)

In [23]:
# Simulating walks for Deepwalk

%%time
g.read_edg(edg_graph_path, weighted=WEIGHTED, directed=DIRECTED)

walks = g.simulate_walks(num_walks=NUM_WALK, walk_length=WALK_LEN)

Thread #  6 progress: |###                      | 9.99 %
Thread #  3 progress: |###                      | 9.99 %
Thread #  7 progress: |###                      | 9.99 %
Thread #  5 progress: |###                      | 9.99 %
Thread #  8 progress: |###                      | 9.99 %
Thread #  0 progress: |###                      | 9.99 %
Thread #  1 progress: |###                      | 9.99 %
Thread #  9 progress: |###                      | 9.99 %
Thread #  4 progress: |###                      | 9.99 %
Thread # 11 progress: |###                      | 9.99 %
Thread #  2 progress: |###                      | 9.99 %
Thread # 10 progress: |###                      | 9.99 %
Thread #  6 progress: |#####                    | 19.99 %
Thread #  3 progress: |#####                    | 19.99 %
Thread #  5 progress: |#####                    | 19.99 %
Thread #  7 progress: |#####                    | 19.99 %
Thread #  9 progress: |#####                    | 19.99 %
Thread #  1 progress: |###

In [24]:
# Asserting that the number of walks equals expected count

assert len(walks) == Node_Count*NUM_WALK

In [18]:
print(walks[0])

['21405226', '21402696', '21403660', '21401562', '21401454', '21401223', '21400508', '1005105', '1004249', '1002544', '1002524', '5100816', '100042795', '5100476', '5100463', '5100577', '100084968', '10100767', '10100052', '10100054', '10100767', '10101219', '10100825', '100040048', '10400618', '7203417', '8000320', '8000040', '8000303', '8000003', '8000351', '100012123', '7900702', '7101466', '7101666', '7101898', '7101926', '7102070', '7101666', '7101819', '7101816', '7101976', '100064427', '7002254', '7001735', '7002066', '7002254', '7005191', '7003686', '7003804', '6902600']


In [26]:

# Training the Deepwalk model using the generated walks

%%time
# model = Word2Vec(sentences=walks, vector_size=128, window=7, workers=-1, negative=10)

model = Word2Vec(walks,  
                 hs=1, 
                 sg = 1,  
                 vector_size=128,  
                 window=5,
                 min_count=1,
                 workers=-1,
                 seed=42)

CPU times: user 9h 11min 54s, sys: 3min 21s, total: 9h 15min 15s
Wall time: 2h 26min 56s


In [27]:
# Saving the trained Deepwalk model
model.save('../Model/deepwalk_graph_embedding_'+GRAPH_FILE_NAME.split('.')[0]+'.model')

In [28]:
# Extracting embedding vectors from the Deepwalk model

pid_set = set(graph['pid_1'].unique()).union(graph['pid_2'].unique())
payload = []
for pid in tqdm(pid_set):
    try:
        payload.append({'pid': pid, 'embedding_vector': model.wv[str(pid)]})
    except:
        print(pid, "Not Exist")
        pass

100%|██████████| 211861/211861 [00:01<00:00, 201291.01it/s]


In [29]:
# Saving the Deepwalk embedding vectors

embedding_df = pd.DataFrame(payload)
embedding_df.to_parquet('../Data/Embedding_Data/deepwalk_embedding_df_'+GRAPH_FILE_NAME.split('.')[0]+'.parquet', index=False)

## Node2vec

In [7]:
# Initializing the node2vec model with parameters for biased random walks

g = node2vec.SparseOTF(p=1, q=0.5, workers=-1, verbose=True, extend=True)

In [8]:
# Simulating biased walks for node2vec4

%%time
g.read_edg(edg_graph_path, weighted=WEIGHTED, directed=DIRECTED)

walks = g.simulate_walks(num_walks=NUM_WALK, walk_length=WALK_LEN)

Thread #  2 progress: |###                      | 9.99 %
Thread #  4 progress: |###                      | 9.99 %
Thread #  8 progress: |###                      | 9.99 %
Thread #  7 progress: |###                      | 9.99 %
Thread #  3 progress: |###                      | 9.99 %
Thread #  9Thread #  progress: |###                      | 9.99 %
 5 progress: |###                      | 9.99 %
Thread #  6 progress: |###                      | 9.99 %
Thread # 10 progress: |###                      | 9.99 %
Thread #  1 progress: |###                      | 9.99 %
Thread # 11 progress: |###                      | 9.99 %
Thread #  0 progress: |###                      | 9.99 %
Thread #  4 progress: |#####                    | 19.99 %
Thread #  2 progress: |#####                    | 19.99 %
Thread #  3 progress: |#####                    | 19.99 %
Thread #  7 progress: |#####                    | 19.99 %
Thread #  8 progress: |#####                    | 19.99 %
Thread #  5 progress: |###

In [9]:
assert len(walks) == Node_Count*NUM_WALK

In [10]:
print(walks[0])

['11500381', '11500310', '11500720', '11500159', '4804295', '5100816', '5100817', '100041309', '5100536', '5100597', '100041309', '5100818', '5100816', '5100875', '5100719', '5100564', '5100577', '100015591', '5100895', '100015591', '5100876', '5100888', '5100860', '5100852', '3601283', '3601605', '3601603', '3600661', '3601495', '3601632', '3601638', '3601015', '3600980', '3600661', '3600937', '3600666', '3600937', '3600661', '3600666', '3601347', '3600182', '3601489', '3601537', '3600937', '3601437', '3601421', '3601286', '3601454', '3600999', '3601514', '2702331']


In [11]:
# Training the node2vec model using the generated biased walks

%%time
model = Word2Vec(walks,  # previously generated walks
                 hs=1,  # tells the model to use hierarchical softmax
                 sg = 1,  # tells the model to use skip-gram
                 vector_size=128,  # size of the embedding
                 window=5,
                 min_count=1,
                 workers=4,
                 seed=42)

CPU times: user 4h 27min 53s, sys: 1min 1s, total: 4h 28min 55s
Wall time: 1h 9min 22s


In [12]:
# Saving the trained node2vec model

model.save('../Model/node2vec_graph_embedding_'+GRAPH_FILE_NAME.split('.')[0]+'.model')

In [13]:
# Extracting embedding vectors from the node2vec model

pid_set = set(graph['pid_1'].unique()).union(graph['pid_2'].unique())
payload = []
for pid in tqdm(pid_set):
    try:
        payload.append({'pid': pid, 'embedding_vector': model.wv[str(pid)]})
    except:
        print(pid, "Not Exist")
        pass

100%|██████████| 211861/211861 [00:01<00:00, 182278.49it/s]


In [14]:
embedding_df = pd.DataFrame(payload)
embedding_df.to_parquet('../Data/Embedding_Data/node2vec_embedding_df_'+GRAPH_FILE_NAME.split('.')[0]+'.parquet', index=False)

In [15]:
embedding_df.head()

,pid,embedding_vector
0,17301504,"[0.44851154, 0.40735474, -0.172185, 0.07374085..."
1,17301505,"[0.47892618, 0.20487863, 0.021507299, -0.62510..."
2,17301506,"[0.25423482, 0.47845665, -0.001093431, 0.03616..."
3,17301507,"[-0.10483544, 0.22859468, -0.16854355, 0.21888..."
4,17301508,"[0.83427334, 0.5198858, 0.011705197, 0.4507888..."
